In [1]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np
from outils import read, keys, load_cn_json, dump_cn_json, 中转数, 数转中, set_char_colors, nice_print

def wrap(s, wrapper="{}", keep_wrapper=False):
    if s:
        return wrapper[0] + s + wrapper[-1]
    if keep_wrapper:
        return wrapper
    return ""

def make_params(params, wrapper="[]", sep=","):
    return wrap(sep.join(params), wrapper)

def wrap_env(name, content, params=[], param_wrapper="[]", param_sep=","):
    out = r"\begin" + wrap(name) + make_params(params, wrapper=param_wrapper, sep=param_sep) + "\n"
    lines = content[:-1].split("\n")  # presume content ends with \n
    for line in lines:
        out += "    " + line + "\n"
    out += r"\end" + wrap(name) + "\n"
    return out

def wrap_method(method, content="", wrapper="{}", keep_wrapper=True, params=[], param_wrapper="[]", param_sep=","):
    return '\\' + method + make_params(params, wrapper=param_wrapper, sep=param_sep) + wrap(content, wrapper=wrapper, keep_wrapper=keep_wrapper)

def zihao(n):
    return wrap_method("zihao", str(n))

def package_update_xcolor(packages, texts):
    xcolor = packages["xcolor"]
    xcolor["defined_colors"] = {}
    for _, text in texts.items():
        if "character_colors" in text:
            for key, val in text["character_colors"].items():
                xcolor["defined_colors"][key] = val
    packages["xcolor"] = xcolor

def make_ctex_env(document_class="ctexbook", document_class_params=("12pt", "UTF-8","openany"), packages={"ctex": [], "titlesec": []}, mainfont="Arial", lineskip="4pt", parskip="10pt", title="标题", author="", date=False, toc=True):
    """make header and footer for ctexbook environment. 
    header
    1. documentclass and parameters 
    2. packages
    3. geometry and fonts
    4. package setups
    5. global typesettings
    6. begin document
    footer
    1. end document
    """
    # ## header ##

    # document class
    header = r"\documentclass"+ make_params(document_class_params) + wrap(document_class) + "\n"
    
    # packages
    packages_str = ""
    for name in packages:
        # print(package)
        package_declarations = ""
        if "declarations" in packages[name]:
            package_declarations = make_params(packages[name]['declarations'])
        packages_str += r"\usepackage" + package_declarations + wrap(name) + "\n"
    # print(packages_str)
    header += packages_str + "\n"

    # geometry <-- geometry package
    if "geometry" in packages:
        geometry = packages["geometry"]
        paper_type = geometry["paper_size"]
        paddings = geometry["paddings"]
        left = paddings["left"]
        right = paddings["right"]
        top = paddings["top"]
        bottom = paddings["bottom"]
        header += wrap_method("geometry", f"{paper_type}paper,left={left},right={right},top={top},bottom={bottom}") + "\n"
    
    # fonts
    header += r"\renewcommand{\footnotesize}{\fontsize{8.5pt}{10.5pt}\selectfont}" + "\n"
    header += wrap_method("setmainfont", mainfont) + "\n"
    header += r"\setCJKmainfont[BoldFont=STZhongsong]{汉字之美仿宋GBK 免费}" + "\n"
    header += r"\xeCJKDeclareCharClass{CJK}{`0 -> `9}" + "\n"  # apply CJK font to numbers
    header += r"\xeCJKsetup{AllowBreakBetweenPuncts=true}" + "\n"  # line alignment

    if "footmisc" in packages:
        footnote_settings_content = "".join(["{\ding{"+str(192+i)+"}}" for i in range(10)])
        footnote_settings = wrap_method("DefineFNsymbols", footnote_settings_content, params=["circled"], param_wrapper="{}")
        header += footnote_settings + "\n"
        header += wrap_method("setfnsymbol", "circled") + "\n"

    # package setups
    # xpinyin
    if "xpinyin" in packages:
        pyr = packages['xpinyin']['ratio']  # size ratio
        vsep = packages['xpinyin']['vsep']  # vertical gap
        vsep_str = "vsep={" + vsep + "}"
        hsep = packages['xpinyin']['hsep']  # horizontal gap
        hsep_str = "hsep={" + f"{hsep} plus {hsep}" + "}"
        header += wrap_method("xpinyinsetup", f"ratio={pyr},{hsep_str},{vsep_str}") + "\n"  # pinyin settings

    # package setups
    # xcolor
    if "xcolor" in packages:
        defcolor_str = ""
        for key, (r, g, b) in packages["xcolor"]["defined_colors"].items():
            rgb_plate = f"{r},{g},{b}"
            defcolor_str += wrap_method("definecolor", key) + r"{RGB}{" + rgb_plate + r"}" + "\n"
        header += defcolor_str + "\n"

    # global typesettings
    # title format
    header += r"\titleformat{\chapter}{\zihao{-1}\bfseries}{ }{16pt}{}" + "\n"
    header += r"\titleformat{\section}{\zihao{-2}\bfseries}{ }{0pt}{}" + "\n"
    header += r"\title" + wrap(r"\zihao{0} \bfseries " + title) + "\n"
    # line and paragraph skips
    header += r"\setlength{\lineskip}{" + lineskip + "}\n"  # skip length after line
    header += r"\setlength{\parskip}{" + parskip + "}\n"  # extra skip for paragraphs 
    # front page format
    if author:  # author format
        header += r"\author{\zihao{2} \texttt" + wrap(author) + "}\n"
    else:
        header += r"\author{}" + "\n"
    if date:  # date format
        header += r"\date{\bfseries\today}" + "\n"
    else:
        header += r"\date{}" + "\n"
    
    # begin document
    header += r"\begin" + wrap("document") + "\n"
    header += r"\maketitle" + "\n"
    if toc:
        header += r"\tableofcontents" + "\n"
    header += r"\newpage" + "\n"
    
    # ## footer ##

    # end document
    footer = r"\end" + wrap("document") + "\n"
    return header, footer


In [2]:
# 打印页面设置：纸号，页边距等
geometry = {}
geometry["paper_size"] = "a5"  # 使用A5纸
paddings = {}  # 页边距
paddings["left"] = "1.4cm"
paddings["right"] = "1.4cm"
paddings["top"] = "2.3cm"
paddings["bottom"] = "2.3cm"
geometry["paddings"] = paddings

# 拼音设置： xpinyin宏包
pinyin = {}
pinyin["ratio"] = "0.5"
pinyin["hsep"] = ".6em"
pinyin["vsep"] = "1em"

## 小学

In [5]:
path_xx = "../src/小学/"

# 打印小学古诗（分层）
packages = {}
packages["ctex"] = []
packages["titlesec"] = []
packages["xeCJK"] = []
packages["fontspec,xunicode,xltxtra"] = []
packages["xpinyin"] = pinyin
packages["xpinyin"]['ratio'] = "0.44"
packages["xpinyin"]['hsep'] = ".6em"
packages["geometry"] = geometry
packages["indentfirst"] = []
packages["pifont"] = []
packages["footmisc"] = {"declarations": ["perpage", "symbol*"]}
lineskip = "24pt"
parskip = "6pt"

### 小学诗歌

In [7]:
def shi_to_tex_str(shi, print_genre=False, authors={}, typesettings={"vspaces": {"after_title": 8, "after_author": 6, "after_content": 6}}):
    # convert structured shi to string ready to use in tex
    out = r"\section{" + shi["title"] + "}\n\n"
    content = ""
    # title = wrap_method("textbf", zihao(3) + " " + shi["title"]) + "\n\n"

    # if print_genre:
    #     title = shi["genre"] + "：" + title
    # content += title
    content += wrap_method("vspace", f"{typesettings['vspaces']['after_title']}pt") + "\n\n"
    author_str = ""
    if shi["author"]:
        author = shi["author"]
        if author in authors:
            author_str += "〔唐代：" + author + "〕\n\n"
        else:
            author_str += "〔" + author + "〕\n\n"
    else:
        author_str += "〔作者不详〕\n\n"
    content += wrap_env("normalsize", "\n" + author_str) + "\n"
    content += wrap_method("vspace", f"{typesettings['vspaces']['after_author']}pt") + "\n\n"
    content += wrap_env("large", "\n" + "\n\n".join([wrap_method("xpinyin*", line) for line in shi["content"]]) + "\n\n") + "\n"
    content = wrap_env("center", content) + "\n"
    content += wrap_method("vspace", f"{typesettings['vspaces']['after_content']}pt") + "\n\n"
    out += content
    return out

In [8]:
shis = load_cn_json(os.path.join(path_xx, "古诗.json"))
output_tex = "古诗集.tex"
title = "小学语文古诗集"
# shis = load_cn_json(os.path.join(path_xx, "唐诗三百首.json"))
# output_tex = "唐诗三百首.tex"
# title = "唐诗三百首"

header, footer = make_ctex_env(packages=packages, title=title, parskip=parskip, lineskip=lineskip)

# 分层
shi_by_level = {}
levels = []
for i in range(10):
    levels.append(f"第{数转中[i+1]}层") 
levels.append("其他")
# print(levels)

typesettings = {"vspaces": {"after_title": 10, "after_author": 8, "after_content": 8}}

for title, shi in shis.items():
    level = shi["level"]
    if level not in shi_by_level:
        shi_by_level[level] = {}
    shi_by_level[level][title] = shi

with open(output_tex, "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for level in levels:
        f.write(r"\chapter" + wrap(level) + "\n\n")
        for title, shi in shi_by_level[level].items():
            f.write(shi_to_tex_str(shi, typesettings=typesettings) + "\n")
    f.write(footer)

### 小学现代文

In [3]:
def read_text(path, format="散文"):
    """Read raw text and formalize to json
    Inputs: 
    path (str): file path to the raw text.
    format (str): format of the text.
    Output:
    out (dict): a jsonifiable dictionary with formalized text.
    Example:
    out["format"]     : format of the text (in the sense of tex printing).
    out["genre"]      : genre and other tags of the text.
    out["content"]    : content of the text. A list of strings.
    out["grade"]      : recommanded student grade (for the purpose of eduation).
    out["title"]      : title of the text.
    out["author"]     : author of the text.
    out["remarks"]    : remarks concerning the text.
    out["footnotes"]  : footnotes of the content of the text.
    out["endnotes"]   : endnotes of the content of the text.
    out["vocabulary"] : vocabulary to learn (for the purpose of eduation).
    """
    lines = read(path)
    out = {}
    title = ""
    if len(lines) and len(lines[0]):
        author = ""
        grade = 0
        footnotes = []
        endnotes = []
        vocabulary = []
        remarks = []
        content = []
        out["format"] = format
        out["genre"] = [format]
        # return lines
        if format in ("散文", "书信", "小说", "剧本"):
            for line in lines:
                line0 = line.strip()
                if line0:
                    if not title:
                        title = line0
                    elif grade < 1 and line.startswith("年级："):
                        grade = int(line0[3:])
                    elif not author and line.startswith("作者："):
                        author = line0[3:]
                    elif line.startswith("备注："):
                        remarks.append(line0[3:])
                    elif line.startswith("注释："):
                        footnotes.append(line0[3:])
                    elif line.startswith("脚注："):
                        footnotes.append(line0[3:])
                    elif line.startswith("尾注："):
                        endnotes.append(line0[3:])
                    elif line.startswith("词汇："):
                        vocabulary.extend(line0[3:].split())
                    else:
                        content.append(line0)
        elif format == "诗歌":
            para = []
            for line in lines:
                line0 = line.strip()
                if line0:
                    if not title:
                        title = line0
                    elif grade < 1 and line.startswith("年级："):
                        grade = int(line0[3:])
                    elif not author and line.startswith("作者："):
                        author = line0[3:]
                    elif line.startswith("备注："):
                        remarks.append(line0[3:])
                    elif line.startswith("注释："):
                        footnotes.append(line0[3:])
                    elif line.startswith("脚注："):
                        footnotes.append(line0[3:])
                    elif line.startswith("尾注："):
                        endnotes.append(line0[3:])
                    elif line.startswith("词汇："):
                        vocabulary.extend(line0[3:].split())
                    else:
                        para.append(line0)
                elif len(para):
                    content.append(para)
                    para = []
            if len(para):
                content.append(para)
        out["title"] = title
        out["author"] = author
        out["content"] = content
        out["remarks"] = remarks
        out["footnotes"] = footnotes
        out["endnotes"] = endnotes
        out["vocabulary"] = vocabulary
        if grade:
            out["grade"] = grade
    return title, out

def footnotes_to_text(out, footnotes=[]):
    """Add footnotes to the text.
    Inputs:
    out (str): text string output to tex.
    foonotes (list of str): list of foonotes to insert into the output text string.
    Output:
    out (str): text string with footnotes inserted.
    """
    if len(footnotes):
        if "footnote" in out:  # add footnotes when the position is provided
            for i, note in enumerate(footnotes):
                out = out.replace("footnote{"+str(i+1)+"}", "footnote{" + note + "}")
        else:
            note_dict = {}
            keybase = "b"  # base string to make keys. Marked keys start with "a", auto-generated keys start with "b".
            i = 0
            for note in footnotes:
                word = ""
                if note.startswith("〔"):
                    word = note.split("〕")[0][1:]
                    key = keybase + str(i+1)
                    note_dict[key] = note
                    i += 1
                elif "〕" in note:  # key is already marked in the text with the format "\apost{a...}".
                    key = note.split("〕")[0].split("〔")[0]
                    note_dict[key] = "".join(note.split(key)[1:])
                # print(word)
                if word:  # find the position to insert footnote and mark
                    nfin = out.find(word) + len(word)
                    out = out[:nfin] + r"\apost{" + key + "}" + out[nfin:]
            for key in note_dict:
                out = out.replace("apost{"+key+"}", "footnote{" + note_dict[key] + "}")
    return out

def text_content_to_tex_str(text, verbose=0, verseprop=0.5):
    """convert the content of a text to text string ready for tex.
    the format varies by genre:
    散文、小说
    书信
    诗歌
    剧本
    """ 
    content = text["content"]
    footnotes = text["footnotes"]
    format = text["format"]
    out = ""
    if format in ("散文", "小说",):
        out = "\n\n".join(content) + "\n"
    elif  format == "书信":
        if verbose and not content[0].endswith("："):
            print("错误：第一行不是抬头")
            return "格式错误\n"
        out = r"\noindent " + content[0] + "\n\n" + wrap_method("vspace", "24pt") + "\n\n"
        toright = False
        toright_content = ""
        for line in content[1:]:
            if line:
                if toright:
                    toright_content += line + "\n\n"
                else:
                    out += line + "\n\n"
            else:
                toright = True
        out += wrap_method("vspace", "36pt") + "\n\n"
        out += wrap_env("flushright", toright_content) + "\n\n"
    elif  format == "诗歌":
        if not isinstance(content[0], list):
            content = [content]
        
        lineskip = " \\\\\n"
        # parskip = "\n" + wrap_method("vspace", "4pt") + "\n\n"
        parskip = "\n\n"
        out = parskip.join([wrap_env("verse", lineskip.join(par) + "\n", params=[str(verseprop)+"\\linewidth"]) for par in content])
    elif format == "剧本":
        name_set = text["characters"]
        for line in content:
            if line.startswith("\\item["):
                name = line.split("]")[0][6:]
                colored_name = r"{\color{" + name_set[name] + r"} " + name + r"}"
                out += "\\item[" + colored_name + "]" + "]".join(line.split("]")[1:])
            elif line.startswith("$"):
                colored_line = line
                for name in name_set:
                    colored_line = colored_line.replace(name, r"{\color{" + name_set[name] + r"} " + name + r"}")
                out += colored_line
            else:
                out += line
            out += "\n\n"
    if len(footnotes):
        if "footnote" in out:  # add footnotes when the position is provided
            for i, note in enumerate(footnotes):
                out = out.replace("footnote{"+str(i+1)+"}", "footnote{" + note + "}")
                # out = ssub(r"footnote\{(\d+?)\}", "footnote{" + note + "}", out, 1)
        else:
            note_dict = {}
            keybase = "b"
            i = 0
            for note in footnotes:
                word = ""
                if note.startswith("〔"):
                    word = note.split("〕")[0][1:]
                    key = keybase + str(i+1)
                    note_dict[key] = note
                    i += 1
                elif "〕" in note:
                    key = note.split("〕")[0].split("〔")[0]
                    note_dict[key] = "".join(note.split(key)[1:])
                # print(word)
                if word:  # find where to insert footnote and mark
                    nfin = out.find(word) + len(word)
                    out = out[:nfin] + r"\apost{" + key + "}" + out[nfin:]
            for key in note_dict:
                out = out.replace("apost{"+key+"}", "footnote{" + note_dict[key] + "}")

    return out

def endnotes_to_str(endnotes, verbose=0, pinyin=False):
    """convert the endnotes to text string ready for tex."""
    out = ""
    notes = ""
    for note in endnotes:
        if pinyin and note.startswith("〔"):  # add pinyin
            suite = note[1:].split("〕")
            notes += "\item " + note[0] + r"\xpinyin*{" + suite[0] + r"}〕" + "〕".join(suite[1:]) + "\n"
        else:
            notes += "\item " + note + "\n"
    if notes:
        out = r"\newpage" + "\n\n" + r"\textbf{注释}：" + "\n\n" + r"\vspace{-1em}" + "\n\n"
        out += wrap_env("itemize", r"\setlength\itemsep{-0.2em}" + "\n" + notes)
    return out

def text_to_tex_str(text, typesettings={"font": {"title": {"size": 2}, "plaintext": {"size": "normalsize"}}, "vspaces": {"after_title": 12, "after_author": 6, "after_content": 6}}):
    """convert a text object to text string ready for tex
    """
    out = ""
    content = ""
    # title_fontsize = typesettings["font"]["title"]["size"]
    # title = wrap_method("textbf", zihao(title_fontsize) + " " + text["title"]) + "\n"
    title = wrap_method("chapter", text["title"]) + "\n\n"
    content += title
    # content = wrap_env("center", content) + "\n"
    # content += wrap_method("vspace", f"{typesettings['vspaces']['after_title']}pt") + "\n\n"
    content += wrap_env(typesettings["font"]["plaintext"]["size"], "\n" + text_content_to_tex_str(text) + "\n")
    # content += wrap_method("vspace", f"{typesettings['vspaces']['after_content']}pt") + "\n\n"
    out += content + "\n\n"
    # out += wrap_method("newpage", keep_wrapper=False) + "\n\n"
    out += endnotes_to_str(text["endnotes"])
    return out

def add_text(texts, title, content, format="散文", tags=[]):
    """Add a text to the dictionary of texts.
    Inputs:
    texts (dict): dictionary of texts. title --> content.
    title (str): title of the text.
    content (dict): content of the text.
    format (str): format of the text.
    tags (list of str): tags to describe the text.
    Output:
    texts: updated dictionary of texts. 
    """
    if len(tags):
        content["genre"] = tags
    if format == "剧本":
        if title not in texts:
            script_keys = []
            for _, text in texts.items():
                if text["format"] == "剧本" and "key" in text:
                    script_keys.append(int(text["key"].split("-")[1]))
            if len(script_keys):
                script_key = "script-" + str(max(script_keys) + 1)
            else:
                script_key = "script-1"
                
        else:
            script_key = texts[title]["key"]
        name_set, color_set = set_char_colors(content["content"], script_key)
        
    texts[title] = content
    if format == "剧本":
        texts[title]["key"] = script_key
        texts[title]["characters"] = name_set
        texts[title]["character_colors"] = color_set
    return texts


In [512]:
packages = {}
packages["ctex"] = []
packages["titlesec"] = []
packages["xeCJK"] = []
packages["verse"] = []
packages["fontspec,xunicode,xltxtra"] = []
packages["xpinyin"] = pinyin
packages["geometry"] = geometry
packages["indentfirst"] = []
packages["pifont"] = []
packages["footmisc"] = {"declarations": ["perpage", "symbol*"]}
lineskip = "24pt"
parskip = "6pt"

In [513]:
texts_xx = load_cn_json("../src/小学/阅读课文.json")

text_format = "散文"
tags = ["寓言", "成语故事"]
title, content = read_text("草稿.tex", format=text_format)

if title:
    print(f"新增课文：{title}")
    texts = add_text(texts_xx, title, content, text_format, tags)
    dump_cn_json("../src/小学/阅读课文.json", texts_xx)

新增课文：叶公好龙


In [516]:
texts_xx = load_cn_json("../src/小学/阅读课文.json")

booktitle = "小学语文课文集萃"
header, footer = make_ctex_env(packages=packages, title=booktitle, parskip=parskip, lineskip=lineskip)
typesettings = {}
typesettings["vspaces"] = {"after_title": 36, "after_author": 16, "after_content": 16}
typesettings["font"] = {"plaintext": {"size": "large"}}

with open("小学现代文阅读课文.tex", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for title, text in texts_xx.items():
        f.write(text_to_tex_str(text, typesettings=typesettings) + "\n")
    f.write(footer)

## 中学

### 现代文

In [4]:
packages = {}
packages["ctex"] = []
packages["titlesec"] = []
packages["xeCJK"] = []
packages["verse"] = []
packages["fontspec,xunicode,xltxtra"] = []
packages["xpinyin"] = pinyin
packages["geometry"] = geometry
packages["indentfirst"] = []
packages["pifont"] = []
packages["enumitem"] = []
packages["footmisc"] = {"declarations": ["perpage", "symbol*"]}
xcolor = {}
xcolor["declarations"] = ["table", "dvipsnames"]
packages["xcolor"] = xcolor

typesettings = {}
typesettings["vspaces"] = {"after_title": 36, "after_author": 16, "after_content": 16}
typesettings["font"] = {"plaintext": {"size": "normalsize"}}

In [437]:
def sort_zs(path):
    lines = read("草稿.tex")

    out = []
    zs = False
    for line in lines:
        if line[0] in "0987654321":
            zs = True
            continue
        if zs:
            parts = line.split("：")
            out.append(f"注释：〔{parts[0]}〕" + "：".join(parts[1:]))
            zs = False
        else:
            newline ="".join([w for w in line if w not in "0987654321"])
            out.append(newline)

    with open("草稿.tex", "w", encoding="utf-8") as f:
        f.writelines(out)

In [528]:
texts_cz = load_cn_json("../src/中学/阅读课文.json")

text_format = "散文"
tags = ["小说", "浪漫主义", "都市童话"]
title, content = read_text("草稿.tex", format=text_format)

if title:
    print(f"新增课文：{title}")
    texts = add_text(texts_cz, title, content, text_format, tags)
    dump_cn_json("../src/中学/阅读课文.json", texts_cz)

新增课文：麦琪的礼物


In [10]:
booktitle = "中学语文课文集萃"
texts_cz = load_cn_json("../src/中学/阅读课文.json")

lineskip = "24pt"
parskip = "6pt"
package_update_xcolor(packages, texts_cz)
header, footer = make_ctex_env(packages=packages, title=booktitle, parskip=parskip, lineskip=lineskip)
with open("中学现代文阅读课文.tex", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for title, text in texts_cz.items():
        f.write(text_to_tex_str(text, typesettings=typesettings) + "\n")
    f.write(footer)

In [6]:
for title, text in texts_cz.items():
    if text["author"] == "鲁迅":
        if "节选" in text["genre"]:
            print(title+ "（节选）")
        else:
            print(title)

从百草园到三味书屋
阿长与山海经
论雷峰塔的倒掉
“友邦惊诧”论
社戏
故乡
藤野先生
孔乙己
中国人失掉自信力了吗
拿来主义
祝福
聪明人和傻子和奴才
记念刘和珍君
《呐喊》自序
药
阿Q正传（节选）


In [6]:
# [title for title in texts_cz if texts_cz[title]['author'] == "鲁迅"]
texts_cz = load_cn_json("../src/中学/阅读课文.json")
grade_count = {}
title_by_grade = {}
title_by_genre = {}
genre_by_grade = {}
for title, text in texts_cz.items():
    g = text["grade"]
    if g not in grade_count:
        grade_count[g] = 0
    if g not in title_by_grade:
        title_by_grade[g] = []
    if g not in genre_by_grade:
        genre_by_grade[g] = {}
    for genre in text["genre"]:
        if genre not in genre_by_grade[g]:
            genre_by_grade[g][genre] = []
        if genre not in title_by_genre:
            title_by_genre[genre] = []
    grade_count[text["grade"]] += 1
    title_by_grade[text["grade"]].append(title)
    for genre in text["genre"]:
        genre_by_grade[g][genre].append(title)
        title_by_genre[genre].append(title)

res = [(k,len(v)) for k, v in title_by_genre.items()]

# alist = res
def printsort_int(alist, rev=False):
    ma = max([b for (_, b) in alist])
    tem = [[] for _ in range(ma+1)]
    for (a, b) in alist:
        tem[b].append(a)
    
    if rev:
        for i, a in enumerate(tem[::-1]):
            if len(a):
                print(ma-i, a)
    else:
        for i, a in enumerate(tem):
            if len(a):
                print(i, a)

printsort_int(res, True)

23 ['散文']
17 ['经典']
15 ['抒情']
14 ['回忆', '小说', '议论文', '说明文']
12 ['自然']
10 ['科普', '诗歌']
8 ['人物', '浪漫主义', '叙事']
7 ['演讲', '倡议', '现实主义']
6 ['景物', '记叙文', '节选', '社会百态']
5 ['纪实', '论述']
4 ['建筑']
3 ['讽刺', '鼓动', '名著', '批判现实主义', '意象']
2 ['四季', '童话', '借物抒情', '动物', '驳论', '议论', '想象']
1 ['悼词', '报告文学', '书信', '友情', '纪实文学', '刘胡兰', '植物', '亲情', '提出问题', '序言', '戏剧', '墙头诗', '新闻稿', '游记', '对话录', '科学', '推理', '辩论']


In [12]:
nice_print(title_by_genre["自然"])

['济南的冬天', '海燕', '春', '看云识天气', '未选择的路']
['繁星', '静寂的园子', '大自然的语言', '向沙漠进军', '万紫千红的花']
['夜', '绿']
